In [1]:
pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spellchecker import SpellChecker
import gc
import os
import logging

In [3]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [4]:
# Ensure that NLTK resources are downloaded
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Setting up stopwords and the spellchecker
stop_words = stopwords.words('english')
spell = SpellChecker()

In [6]:
df = pd.read_csv('/kaggle/input/song-generator/english_lyrics_df.csv')

In [7]:
df.shape

(3374943, 3)

In [8]:
df.head()

artist              title  \
0    Cam'ron          Killa Cam   
1      JAY-Z         Can I Live   
2   Fabolous  Forgive Me Father   
3    Cam'ron       Down and Out   
4  Lil Wayne             Fly In   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [9]:
# Check for missing values
print(df.isnull().sum())

artist      0
title     127
lyrics      0
dtype: int64


In [10]:
# Check for duplicate rows
print(df.duplicated().sum())

3


In [11]:
# Display a random sample of 5 lyrics entries
print(df['lyrics'].sample(5))

1445885                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              [Chorus]\nThere's a world outside my window\nThat I just can't bear\nBlues held me up against the wall\nFor way too long\nMomma said I drink too much, babe\nCan't hold it down\nBlues held me up against the wall\nFor way too long\n\n[Verse]\nCan't stop me now, babe\nGot it running through my veins\nHad my back against the wall\nFor way too long\nSing blues and fold some rhymes\n'Til the day comes arou

In [12]:
# Define a threshold for maximum allowed characters in lyrics
MAX_CHAR_THRESHOLD = 1000  # Adjust this value based on your analysis

# Filter the DataFrame to remove entries with lyrics exceeding the threshold
filtered_df = df[df['lyrics'].str.len() <= MAX_CHAR_THRESHOLD]

print(f"Original DataFrame had {len(df)} entries, filtered DataFrame has {len(filtered_df)} entries.")

Original DataFrame had 3374943 entries, filtered DataFrame has 1239736 entries.


In [13]:
filtered_df.head(5)

,artist,title,lyrics
160,Domo Genesis,Dreams,"Now if I see it in my dreams\nIs it for real? Somebody tell me what it means\nSometimes it ain't really what it seems\nCause this could all really be a dream\nIs it a dream? Is it a dream?\nAm I living what I'm seeing in my sleep?\nOr is my imagination tripping off some weed?\nIs this all the shit I really need?\nIn my dream, is this a dream?\nWill my dreams make it past here?\nOr will I always try to do better than last year?\nI'm not afraid of what the future brings\nBut this a dream, a vivid dream, it's in my dreams\nI already see it in my dreams"
208,Bizzy Bone,A Thugz Prayer,"[Bizzy Bone]\nI am not John, I am not Jesus\nAnd I could never be Judas\nI can't be a harlot, I could never be a dragon\nMy name, is Bryon\nI am a slave of Allah\nThe master, of creation\nThe creator of alllllllll that Allah needs\nThe birds, the bees, the trees\nAnd all that exists\nAnd if the Big Bang Theory is actually true\nThen you must explain to me, explain to me\nExplain to me creation\nAnd explain to me nothing"
400,2Pac,St. Ides Liquor Commercial 1993,"[Intro: 2Pac Talking]\nHA HA HA HA HA\nYou got St. Ides?\nOur friends getting it though\nIt's it..\nSt. - Ides\n\n[2Pac]\nNobody knows the trouble that brother sees\nMobbing with the homies cause they're true Gs\nCount cash and, hands on the Ides cold glass\nAnd yet.. waiting for my homie to pass me\nThe S-T-Crooked-Ides\nTook the sipping out, had to take another\nBefore I pack a ten for my brother\nThug for life, a G 'til they bury me\nBusting, I don't be trusting these tricks\nSo they done worry me\nNow, I'm a well known figure, getting paid\nCame a long way from the old days\nI gotta get mine, cause in my mind;\nPour some liquor on the curb for my dogs\n[Outro: 2Pac Talking w/ Echoes]\nSt. Ides\nRight?"
434,Charizma & Peanut Butter Wolf,Talk About A Girl,"Errr why what happened?\nOh okay\nOkay okay, I give in\nYou got a friend? Then meet me and my man arooound ten\nYou see I play a Too $hort tape\nJust to let you tricky little girls know I'm not fake\nA little anxious school girl get comfty\nDim the lights and unplug the phone cause it bugs me\nI'm wearin' Timbs with a .22 at my waist\nApple juice is cold and I'm in the place\nSometimes I think about you (*echos*)\nSometimes I think about your whole crew\nI got style, so I relax\nKnowin' I wanna dive in it, it'll take twenty laps\nHop in my mom car and lets take a ride\nThen maybe you will realize\nI know she saw me (*echos*)"
607,Loaded Lux,Gully TV DVD Freestyle,"I been rocking since the cradle\nBarfight, my jaw tight, my chin is stable\nMy kenel cable won't be stripped by any label\nWe gots to sticks together like chinks at the dinner table\nDope slingers, the throat stranglers\nShows, hoes hang out the rows like boat anchors\nCoat hangers, I'm in the arms, launching on no-namers\nI been battling scarecrows, this shit's been a no-brainer\nHe hoping bail come through\nThe jail feel like shampoo, that was the cell son blew\nThe brick get frail, piff go stale\nIt's like a nigga hustling always seem to tip with the scale\nI got scripts in the mail, paint the slum block vivid\nWhere that young Pac kick it there's always some cop frisking\nHeavy metal banging and we don't let the punks rock\nAnd a nigga with the jumpshot clicking always the gunshot victim"


In [14]:
def preprocess_text(text):
    # Lowercase to normalize case
    text = text.lower()

    # Removing specific annotations and preserving lyrical punctuation
    text = re.sub(r"\[.*?\]|\(.*?\)", "", text)  # Remove annotations like [Chorus] or (Verse)
    text = re.sub(r'[^\w\s,!?]', '', text)  # Preserve commas, exclamations, and question marks

    # Compact multiple whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenization and join back without filtering for stopwords or correcting spelling
    tokens = text.split()  # Using simple split to preserve structure and repetitions

    # Return joined tokens as the final preprocessed text
    return ' '.join(tokens)

In [15]:
def process_batches(df, output_file, batch_size=1000):
    df.reset_index(drop=True, inplace=True)
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
        last_processed_idx = existing_df.tail(1).index.item() * batch_size
    else:
        last_processed_idx = -1

    num_batches = (len(df) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min(start_idx + batch_size, len(df))
        if start_idx <= last_processed_idx:
            continue
        
        batch = df.iloc[start_idx:end_idx].copy()
        batch['lyrics'] = batch['lyrics'].apply(preprocess_text)  # No row_number is passed here
        
        batch.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))
        del batch
        gc.collect()


In [16]:
# sample_df = df.sample(n=3000000, random_state=42)  # Sample for initial testing
process_batches(filtered_df, 'English_preprocessed_data.csv', batch_size=100000)

In [17]:
output_file_filtered = pd.read_csv('/kaggle/working/English_preprocessed_data.csv')

In [18]:
output_file_filtered.head()

,artist,title,lyrics
0,Domo Genesis,Dreams,"now if i see it in my dreams is it for real? somebody tell me what it means sometimes it aint really what it seems cause this could all really be a dream is it a dream? is it a dream? am i living what im seeing in my sleep? or is my imagination tripping off some weed? is this all the shit i really need? in my dream, is this a dream? will my dreams make it past here? or will i always try to do better than last year? im not afraid of what the future brings but this a dream, a vivid dream, its in my dreams i already see it in my dreams"
1,Bizzy Bone,A Thugz Prayer,"i am not john, i am not jesus and i could never be judas i cant be a harlot, i could never be a dragon my name, is bryon i am a slave of allah the master, of creation the creator of alllllllll that allah needs the birds, the bees, the trees and all that exists and if the big bang theory is actually true then you must explain to me, explain to me explain to me creation and explain to me nothing"
2,2Pac,St. Ides Liquor Commercial 1993,"ha ha ha ha ha you got st ides? our friends getting it though its it st ides nobody knows the trouble that brother sees mobbing with the homies cause theyre true gs count cash and, hands on the ides cold glass and yet waiting for my homie to pass me the stcrookedides took the sipping out, had to take another before i pack a ten for my brother thug for life, a g til they bury me busting, i dont be trusting these tricks so they done worry me now, im a well known figure, getting paid came a long way from the old days i gotta get mine, cause in my mind pour some liquor on the curb for my dogs st ides right?"
3,Charizma & Peanut Butter Wolf,Talk About A Girl,"errr why what happened? oh okay okay okay, i give in you got a friend? then meet me and my man arooound ten you see i play a too hort tape just to let you tricky little girls know im not fake a little anxious school girl get comfty dim the lights and unplug the phone cause it bugs me im wearin timbs with a 22 at my waist apple juice is cold and im in the place sometimes i think about you sometimes i think about your whole crew i got style, so i relax knowin i wanna dive in it, itll take twenty laps hop in my mom car and lets take a ride then maybe you will realize i know she saw me"
4,Loaded Lux,Gully TV DVD Freestyle,"i been rocking since the cradle barfight, my jaw tight, my chin is stable my kenel cable wont be stripped by any label we gots to sticks together like chinks at the dinner table dope slingers, the throat stranglers shows, hoes hang out the rows like boat anchors coat hangers, im in the arms, launching on nonamers i been battling scarecrows, this shits been a nobrainer he hoping bail come through the jail feel like shampoo, that was the cell son blew the brick get frail, piff go stale its like a nigga hustling always seem to tip with the scale i got scripts in the mail, paint the slum block vivid where that young pac kick it theres always some cop frisking heavy metal banging and we dont let the punks rock and a nigga with the jumpshot clicking always the gunshot victim"
